<a href="https://colab.research.google.com/github/COMS-BC3159-SP24/colabs/blob/main/HelloCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello CUDA

A CPU (Central Processing Unit) and a GPU (Graphics Processing Unit) are both types of processors, but they are designed for different types of workloads.

A CPU is designed to handle a wide variety of tasks, such as running the operating system, executing application programs, and performing complex calculations. A CPU typically has a small number of cores, which are optimized for sequential execution of instructions, and a large cache memory to hold frequently used data.

On the other hand, a GPU is designed to handle large amounts of data in parallel. A GPU has a large number of cores, which are optimized for executing the same instruction on multiple pieces of data in parallel. GPUs also have a large amount of memory called VRAM (Video RAM) that holds the data that the cores are working on.

In general, a CPU is better suited for tasks that require sequential execution of instructions and complex branching operations, while a GPU is better suited for tasks that can be parallelized, such as image processing, scientific simulations, and machine learning.

When a program is executed on a CPU it's called serial execution, while when a program is executed on a GPU it's called parallel execution. A GPU can perform the same operation on thousands of data points at the same time, this means that a GPU can perform certain tasks much faster than a CPU, specially when the task can be parallelized.

In recent years, with the rise of deep learning and AI, the use of GPUs for general purpose computation (GPGPU) has increased, and many libraries and frameworks for deep learning such as TensorFlow, PyTorch, Caffe and others are optimized to run on GPUs to accelerate the computation.

In [ ]:
# make sure CUDA is installed
!nvcc --version

In [ ]:
# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

In [ ]:
# CUDA in Jupyter helpers
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

The CUDA programming model is based on the concept of dividing a computation into small, independently executing units called threads. These threads are organized into groups called thread blocks. A thread block is a collection of threads that can cooperatively share data through shared memory and synchronize their execution through barrier instructions.

**Threads**: A thread is the basic unit of execution in CUDA. Each thread has its own program counter, registers, and stack, and can execute its own instructions independently of other threads. In CUDA, threads are organized into a one, two or three-dimensional grid of thread blocks, and each thread is identified by a unique thread index within its block.

**Thread Blocks**: A thread block is a collection of threads that can cooperatively share data through shared memory and synchronize their execution through barrier instructions. Thread blocks are organized into a one, two or three-dimensional grid of thread blocks, and each thread block is identified by a unique block index within the grid. Threads within a block can communicate through shared memory and synchronize their execution by executing barrier instructions.

Thread blocks are launched on the GPU in a kernel launch call, the kernel launch syntax `<<<dimGrid, dimBlock>>>` is used to specify the number of blocks and the number of threads in each block.

Threads within a block are executed in parallel, and each thread can be thought of as a separate execution unit that can run its own instruction stream. Threads in different blocks can also run in parallel and are executed independently of one another.

In CUDA, the programmer has to decide on the number of blocks and the number of threads per block, this is called the thread hierarchy. The thread hierarchy is used to balance the computation and memory accesses with the number of available CUDA cores and the amount of shared memory. Choosing the right thread hierarchy is crucial for the performance of the CUDA program.

## CUDA Hello World

Here's a breakdown of the key elements of this example:

`#include <cuda_runtime.h>`: This is the header file for the CUDA runtime library, which provides the functionality needed to run CUDA kernels on the GPU.

`__global__ void helloWorldKernel()`: This function is a CUDA kernel, which runs on the GPU. The `__global__` keyword indicates that this function is a kernel that can be executed on the GPU. Similarly the `__host__` before the main function says that it runs on the CPU. There is also a `__device__` specifier you can use for GPU helper functions that ONLY get called by other GPU functions.

`helloWorldKernel<<<1, 1>>>()`: This line launches the kernel on the GPU. The triple angle brackets `<<< >>>` is called the kernel launch syntax and it's used to specify the configuration of the kernel launch. In this case, we're launching 1 block of 1 thread.

`cudaDeviceSynchronize()`: This function waits for all the work in the GPU to finish before it allows the CPU code to continue. This is to ensure that the kernel has finished executing before the program exits. This is VERY VERY SLOW and so you only want to synchronize the whole device when you absolutely need to.

In [ ]:
%%cuda
#include <cstdio>
#include <cuda_runtime.h>

// This function runs on the GPU
__global__ void helloWorldKernel() {
    printf("Hello World!\n");
}

// This function runs on the CPU
__host__
int main() {
    // Launch the kernel on the GPU
    helloWorldKernel<<<1, 1>>>();

    // Wait for the kernel to finish executing
    cudaDeviceSynchronize();

    return 0;
}

## Adding in Memory

Lets now explore a CUDA C++ program that adds two arrays of integers.

The program defines a kernel function called `add` that takes four arguments: two input arrays, `a` and `b`, one output array, `c`, and the number of elements in the arrays, `n`. The kernel function uses the built-in `threadIdx.x` variable to determine the index of the thread that is executing the kernel. Each thread adds the corresponding element from `a` and `b` and stores the result in `c`.

In order to handle the fact that there is seperate memory on the **device** (GPU) and **host** (CPU) we need to use the GPU analog to `malloc`, `cudaMalloc`. Do note that while `cudaMalloc` is used on the host it returns a pointer to device memory and it will segfault if you dereference that point in a host function.

With that said, in the main function, we define three arrays, `h_a`, `h_b` and `h_c` (with `h_` indicating it is host memory by convention) and initialize the arrays with some values. Next, we allocate space for three device arrays, `d_a`, `d_b`, and `d_c` using the `cudaMalloc` function.

We then use the `cudaMemcpy` function to copy the data from the host arrays to the device arrays. This is critical as again the memory is completely seperate and we need to ship it over the I/O Bus. As you might imagine this is very slow! And so the idea with using the GPU efficiently is to ship AS MUCH AS POSSIBLE at once!

To invoke the kernel, we use the `add<<<1, n>>>(d_a, d_b, d_c, n);` syntax, where the number of blocks and the number of threads per block are specified by `1` and `n`, respectively.

We then use `cudaMemcpy` again to copy the data from the device array back to the host array.

Finally, we print the contents of the host array and free the device memory using `cudaFree`.

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

__global__
void add(int *d_a, int *d_b, int *d_c, int n){
    // this if statement makes sure we don't go out of bounds of the array
    int i = threadIdx.x;
    if (i < n){
        d_c[i] = d_a[i] + d_b[i];
    }
    // a better way to do this would be with a for loop in case it is not
    //   launched with enough threads to cover all "n" computations
    // for (int i = threadIdx.x; i < n; i += blockDim.x){
    //     d_c[i] = d_a[i] + d_b[i];
    // }
}

__host__
int main(){

    // constants and host memory
    const int n = 10;
    int h_a[n], h_b[n], h_c[n];

    // Initialize the arrays
    for (int i = 0; i < n; i++){
        h_a[i] = i;
        h_b[i] = i * 2;
    }

    // device memory
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, n * sizeof(int));
    cudaMalloc(&d_b, n * sizeof(int));
    cudaMalloc(&d_c, n * sizeof(int));

    // copy from host to device
    cudaMemcpy(d_a, h_a, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaDeviceSynchronize();

    // run the kernel
    add<<<1, n>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();

    // copy the memory back
    cudaMemcpy(h_c, d_c, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();

    // Print the results
    for (int i = 0; i < n; i++){
        std::cout << h_c[i] << " ";
    }
    std::cout << std::endl;

    // free the allocated memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}


## Synchronizations
Synchronization is crucial in CUDA C++ to ensure that threads within a GPU kernel or a block coordinate their activities properly. Without synchronization, concurrent threads can access and modify shared data in an unpredictable and inconsistent manner, leading to incorrect results and even program crashes. Two key concepts for this may occur are as follows:

1. *Data Consistency*: In parallel computing, multiple threads often work together to process data. Without synchronization, these threads may access the same data simultaneously, leading to data races and inconsistent results.

2. *Order of Execution*: CUDA threads are executed asynchronously and can be scheduled in any order by the GPU. Synchronization ensures that threads follow a specific order of execution, which is crucial for algorithms that depend on specific thread coordination.

In the following code examples we'll show why using synchronization is neccessary!

Lets revisit our Hello World example, but this time with the `cudaDeviceSynchronize();` commented out! Run the code, what happens?

In [ ]:
%%cuda
#include <cstdio>
#include <cuda_runtime.h>

// This function runs on the GPU
__global__ void helloWorldKernel() {
    printf("Hello World!\n");
}

// This function runs on the CPU
__host__
int main() {
    // Launch the kernel on the GPU
    helloWorldKernel<<<1, 1>>>();

    // Wait for the kernel to finish executing
    // cudaDeviceSynchronize();

    return 0;
}

Where did the output go? Well what happend is that the kernel launched and while it was running on the GPU, the CPU code kept going and hit the return and exited the program! So the process excited before the `printf` could occur! Try to uncomment that line and run it again. Now you should see the text!

Now imagine you are writing a CUDA program to increment a value `n` times in parallel. In this example we neglect to leverage any synchronization mechanisms. Run the code below to see what happens!

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

__global__
void incrementValue(int *d_val, int n) {
    for (int tid = threadIdx.x; tid < n; tid += blockDim.x){
        *d_val += 1;
    }
}

__host__
int main() {
    const int n = 1000;

    // initialize the value to 0 and copy onto the GPU
    int h_val = 0;
    int *d_val; cudaMalloc((void**)&d_val, sizeof(int));
    cudaMemcpy(d_val, &h_val, sizeof(int), cudaMemcpyHostToDevice);

    // run the kernel
    incrementValue<<<1, n>>>(d_val, n); // Launch kernel with n threads
    cudaDeviceSynchronize();

    // copy the result back
    cudaMemcpy(&h_val, d_val, sizeof(int), cudaMemcpyDeviceToHost);

    // display the result
    std::cout << h_val << " ";

    cudaFree(d_val);
    return 0;
}

Yikes! That's a problem! **What if we instead use some atomics?** Note that all we are changing is the `+=` into an `atomicAdd`!

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

__global__
void incrementValueAtomic(int *d_val, int n) {
    for (int tid = threadIdx.x; tid < n; tid += blockDim.x){
        atomicAdd(d_val,1);
    }
}

__host__
int main() {
    const int n = 1000;

    // initialize the value to 0 and copy onto the GPU
    int h_val = 0;
    int *d_val; cudaMalloc((void**)&d_val, sizeof(int));
    cudaMemcpy(d_val, &h_val, sizeof(int), cudaMemcpyHostToDevice);

    // run the kernel
    incrementValueAtomic<<<1, n>>>(d_val, n); // Launch kernel with n threads
    cudaDeviceSynchronize();

    // copy the result back
    cudaMemcpy(&h_val, d_val, sizeof(int), cudaMemcpyDeviceToHost);

    // display the result
    std::cout << h_val << " ";

    cudaFree(d_val);
    return 0;
}

## Your Turn!

Can you finish the implementation below to design code to preform a vector reduction? Aka summing an array of length `n` of all `1`s using `1` block of `n` threads.

*Note: the code assumes the solution will be put in `d_vals[0]`.*

*Hint: I made `n` a power of 2 to keep things simple for you in terms of bookkeeping. But you probably want to keep track of how many threads you want to use or how many cells offset you want to sum etc. as you loop!*

In [ ]:
%%cuda
#include <iostream>
#include <cuda_runtime.h>

// sums d_vals and places the result in d_vals[0]
__global__
void vectorReduction(int *d_vals, int n) {
    /**********************
     *                    *
     *        TODO        *
     *                    *
     **********************/
}

__host__
int main() {
    const int n = 64;

    // initialize the values to 1 and copy onto the GPU
    int h_vals[n]; for(int i = 0; i < n; i++){h_vals[i] = 1;}
    int *d_vals; cudaMalloc((void**)&d_vals, n*sizeof(int));
    cudaMemcpy(d_vals, h_vals, n*sizeof(int), cudaMemcpyHostToDevice);

    // run the kernel
    vectorReduction<<<1, n>>>(d_vals, n); // Launch kernel with n threads
    cudaDeviceSynchronize();

    // copy the result back
    cudaMemcpy(h_vals, d_vals, sizeof(int), cudaMemcpyDeviceToHost);

    // display the result
    std::cout << h_vals[0] << " ";

    cudaFree(d_vals);
    return 0;
}

**Solution Below -- Don't Scroll Down Until You Are Done**

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

Note that what we are doing here is taking half of the size of the array (`n/2`) and adding that much offset in parallel over threads. Then we are dividing the size by two for the next step and exiting when we are at size 1! Copy and paste the solution in above and see what happens when you vary `n`? For example at `64` you should see it return `64` but at 70 it should also return `64`. Why? How could you fix it? Try to think about what you would want to modify!

In [ ]:
__global__
void vectorReduction(int *d_vals, int n) {
    while(n > 1){
        for(int i = threadIdx.x; i < n/2; i += blockDim.x){
            d_vals[i] += d_vals[i + n/2];
        }
        n /= 2;
    }
}

**Solution Below -- Don't Scroll Down Until You Are Done**

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

We are being foiled by integer math! `n/2` will allways round down to the nearest whole number! So sometimes we miss `1` that we need to add back in when `n` is odd (note that `n%2` is the remainder operator and will return a `1` iff `n` is odd)! BUT we need to be careful to make sure ONLY ONE THREAD adds in that extra value or we are going to overcount things now (hence the check for the `threadIdx.x == 0`)!

In [ ]:
__global__
void vectorReduction(int *d_vals, int n) {
    while(n > 1){
        for(int i = threadIdx.x; i < n/2; i += blockDim.x){
            d_vals[i] += d_vals[i + n/2];
        }
        __syncthreads();
        if (n%2 && threadIdx.x == 0){d_vals[0] += d_vals[n-1];}
        __syncthreads();
        n /= 2;
    }
}